In [ ]:
#python ./spert.py train --config configs/train_reduced.conf

In [2]:
import transformers
import torch
import pandas as pd

### Getting the texts that are not in labeled data, separating into 2 parts

In [3]:
df_labels = pd.read_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/annotation/final_labels.csv', index_col=[0])

In [33]:
df_labels = df_labels[df_labels['Dehumanization']!='не можу визначитись з правильною відповіддю'].copy()

In [35]:
df_labels['label'] = df_labels['Dehumanization'].apply(lambda x: 0 if x=='ні' else 1)

In [36]:
df_labels

,Emotion,Dehumanization,Mention,External ID,Created By,text,rating,label
1219,"так, присутня негативна",так,так,row_0.txt,snizannabotvin@gmail.com,"Всвязи с этим немного поправлю коллег ⤵️ ""Они...",4,1
1218,"ні, оцінка не присутня",ні,ні,row_1.txt,snizannabotvin@gmail.com,Литературный критик Галина Юзефович о новом ро...,4,0
1591,"так, присутня негативна",так,так,row_10.txt,snizannabotvin@gmail.com,Почему на базах неонацистов стоят языческие ис...,4,1
1198,"так, присутня негативна",так,так,row_100.txt,snizannabotvin@gmail.com,Группа добровольцев-медиков из Чеченской Респу...,4,1
3247,"ні, оцінка не присутня",ні,так,row_1000.txt,tutovadesign@gmail.com,"ВСУшники, переходите на сторону добра, у нас т...",5,0
...,...,...,...,...,...,...,...,...
3613,"ні, оцінка не присутня",ні,так,row_995.txt,tutovadesign@gmail.com,Утренний брифинг Минобороны России: ▪️ россий...,5,0
3612,"так, присутня негативна",ні,так,row_996.txt,tutovadesign@gmail.com,И понеслась мазепинщино-петлюровщино-бандеровщ...,5,0
4121,"так, присутня негативна",ні,так,row_997.txt,yevhen.marchenko91@gmail.com,Наш соратник по русскому движению Алексей Сели...,3,0
4120,"ні, оцінка не присутня",так,так,row_998.txt,yevhen.marchenko91@gmail.com,Хорошее видео от 4 бригады НМ ЛНР https://t.me...,3,1


In [37]:
dehumanizing_target_sequences = ['рейх', 'нацист', 'питек', 'бандерло', 'свино', 'вермахт', 'питек']

In [56]:
import re

def contains_target_sequence(text, target_sequences):
    for seq in target_sequences:
        if re.search(seq, text, re.IGNORECASE):
            return True
    return False

In [57]:
def split_test_data_by_target_sequences(X, y, text_columns, target_sequences):
    contains_seq = X.apply(lambda row: any(contains_target_sequence(row[col], target_sequences) for col in text_columns), axis=1)
    X_pos, y_pos = X[contains_seq], y[contains_seq]
    X_neg, y_neg = X[~contains_seq], y[~contains_seq]

    return X_pos, y_pos, X_neg, y_neg

In [40]:
# excluding the rows that were labeled for SpERT training

In [43]:
df_labels_CoNLL04 = pd.read_json('/Users/katerynaburovova/PycharmProjects/dehumanization/annotation/CoNLL04_annotation/SpERT_dataset/export_400samples.json')

In [47]:
exclusion_list = df_labels_CoNLL04['External ID'].to_list()

In [48]:
exclusion_list

['row_1566.txt',
 'row_1565.txt',
 'row_1564.txt',
 'row_1563.txt',
 'row_1562.txt',
 'row_1561.txt',
 'row_1560.txt',
 'row_1559.txt',
 'row_1558.txt',
 'row_1557.txt',
 'row_1556.txt',
 'row_1555.txt',
 'row_1554.txt',
 'row_1553.txt',
 'row_1552.txt',
 'row_1551.txt',
 'row_1550.txt',
 'row_1549.txt',
 'row_1548.txt',
 'row_1547.txt',
 'row_1546.txt',
 'row_1545.txt',
 'row_566.txt',
 'row_565.txt',
 'row_564.txt',
 'row_563.txt',
 'row_562.txt',
 'row_561.txt',
 'row_560.txt',
 'row_559.txt',
 'row_558.txt',
 'row_557.txt',
 'row_555.txt',
 'row_554.txt',
 'row_1544.txt',
 'row_1543.txt',
 'row_1542.txt',
 'row_556.txt',
 'row_1541.txt',
 'row_1540.txt',
 'row_553.txt',
 'row_552.txt',
 'row_551.txt',
 'row_550.txt',
 'row_549.txt',
 'row_548.txt',
 'row_547.txt',
 'row_1539.txt',
 'row_1538.txt',
 'row_1537.txt',
 'row_1536.txt',
 'row_1535.txt',
 'row_1534.txt',
 'row_1533.txt',
 'row_1532.txt',
 'row_1531.txt',
 'row_1530.txt',
 'row_1529.txt',
 'row_1528.txt',
 'row_1527.txt',


In [49]:
df_clean_labels = df_labels[~df_labels['External ID'].isin(exclusion_list)].copy()

In [64]:
X_test_pos, y_test_pos, X_test_neg, y_test_neg = split_test_data_by_target_sequences(df_clean_labels[['text']], df_clean_labels['label'], ['text'], dehumanizing_target_sequences)

In [68]:
import spacy

nlp = spacy.load("ru_core_news_sm")

In [69]:
def tokenize_text(text):
    return [token.text for token in nlp(text)]

In [74]:
def get_SpERT_formatted_input(X):
    formatted_texts = []

    for text in X.text:
        tokens = tokenize_text(text)
        formatted_text = {"tokens": tokens}
        formatted_texts.append([formatted_text, tokens, text])

    return formatted_texts

In [135]:
%%time
X_test_pos_formatted = get_SpERT_formatted_input(X_test_pos)

CPU times: user 8.88 s, sys: 139 ms, total: 9.02 s
Wall time: 9.3 s


In [136]:
%%time
X_test_neg_formatted = get_SpERT_formatted_input(X_test_neg)

CPU times: user 16.4 s, sys: 160 ms, total: 16.6 s
Wall time: 16.8 s


In [137]:
X_test_neg_list_of_dicts = []
for doc in X_test_neg_formatted:
    X_test_neg_list_of_dicts.append(doc[0])

In [138]:
X_test_pos_list_of_dicts = []
for doc in X_test_pos_formatted:
    X_test_pos_list_of_dicts.append(doc[0])

In [139]:
import json

file_path_neg_one_sample = '/Users/katerynaburovova/PycharmProjects/dehumanization/spert/data/datasets/test_datasets/test_neg.json'

with open(file_path_neg_one_sample, 'w', encoding='utf-8') as f:
    json.dump(X_test_neg_list_of_dicts, f, ensure_ascii=False, indent=4)

In [140]:
file_path_pos_one_sample = '/Users/katerynaburovova/PycharmProjects/dehumanization/spert/data/datasets/test_datasets/test_pos.json'

with open(file_path_pos_one_sample, 'w', encoding='utf-8') as f:
    json.dump(X_test_pos_list_of_dicts, f, ensure_ascii=False, indent=4)

In [141]:
file_n_path = '/Users/katerynaburovova/PycharmProjects/dehumanization/spert/data/predictions_neg.json'

with open(file_n_path, 'r') as file:
    data_neg = json.load(file)

In [149]:
len(data_neg[0]['entities'])

0

In [142]:
file_p_path = '/Users/katerynaburovova/PycharmProjects/dehumanization/spert/data/predictions_pos.json'

with open(file_p_path, 'r') as file:
    data_pos = json.load(file)

In [151]:
for sentence in data_neg:
    if len(sentence['entities'])==0:
        sentence['pred'] = 0
    else:
        sentence['pred'] = 1

In [154]:
for sentence in data_pos:
    if len(sentence['entities'])==0:
        sentence['pred'] = 0
    else:
        sentence['pred'] = 1

In [163]:
X_test_pos.reset_index(inplace = True, drop=True)

In [168]:
X_test_pos.iloc[13]

text    Мобики 2-го батальона 56-й бригады ВСУ жалуютс...
Name: 13, dtype: object

In [158]:
data_pos[13]

{'tokens': ['Мобики',
  '2-го',
  'батальона',
  '56-й',
  'бригады',
  'ВСУ',
  'жалуются',
  ',',
  'что',
  'их',
  'без',
  'подготовки',
  'и',
  'соответствующей',
  'экипировки',
  'кинули',
  'на',
  'Пески',
  ',',
  'где',
  'у',
  'хохловермахта',
  'посыпался',
  'фронт'],
 'entities': [{'type': 'HIGH_UH_LOW_NH', 'start': 21, 'end': 22}],
 'relations': [],
 'pred': 1}

In [169]:
pos_predictions = [sentence['pred'] for sentence in data_pos]

In [174]:
neg_predictions = [sentence['pred'] for sentence in data_neg]

In [172]:
from sklearn.metrics import classification_report

In [173]:
report = classification_report(y_test_pos, pos_predictions, target_names=['class_0', 'class_1'])
print(report)

              precision    recall  f1-score   support

     class_0       0.29      0.48      0.36        60
     class_1       0.97      0.93      0.95       967

    accuracy                           0.90      1027
   macro avg       0.63      0.70      0.65      1027
weighted avg       0.93      0.90      0.91      1027



In [175]:
report = classification_report(y_test_neg, neg_predictions, target_names=['class_0', 'class_1'])
print(report)

              precision    recall  f1-score   support

     class_0       0.87      0.94      0.90      1621
     class_1       0.59      0.39      0.47       386

    accuracy                           0.83      2007
   macro avg       0.73      0.66      0.68      2007
weighted avg       0.81      0.83      0.82      2007



## Sampling the timedata from the dataset

1. read timestep data
2. sample the unedited texts
3. get random sentences from selected text
4. save this df with dates, authors
5. write into df